## About the dataset

The dataset consists of 4 columns: product category (e.g. headsets, cell phones etc.), review title, review content and rating. The rating is a numerical type that can take one of the following value: 1, 2, 3, 4, 5. The value of 1 is the worst score, the value of 5 is the best score. The data is not cleaned. It need to be preprocessed for building models

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

import tensorflow
import keras
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K
from keras.layers.embeddings import Embedding



ModuleNotFoundError: No module named 'nltk'

In [ ]:
import tensorflow as tf
print(tf.__version__)

: 

In [ ]:
raw_data = pd.read_csv('/home/suraj/ClickUp/Jan-Feb/DataScience_ML_DL_Projects/MachineLearning/Ecommerce_Product_Review/sentiment_analysis/data/ebay_reviews.csv')
raw_data.head()

Let's clean the dataset by following steps

1. Bringing rating to -1,0 and 1 scale where 0 is neutral, posiitive being 4 and 5 and 1,2 being negative.

2. Removing duplicate words, stop words, duplicate reviews and applying stemmer.

Preprocessing reference - https://www.kaggle.com/code/wojtekbonicki/text-data-cleaning-using-user-defined-transformers

In [ ]:
cols = ['review_title', 'review_content', 'rating']


Dataset transformations via https://scikit-learn.org/stable/data_transforms.html

In [ ]:
class DuplicatesRemover(BaseEstimator, TransformerMixin):
    #Transformer to remove duplicate reviews
    def fit(self,X, y=None):
        return self
    
    def transform(self,X):
        X2 = X.copy()
        #indices of duplicated reviews
        duplicate_idx = X2.duplicated()
        X2= X2[~duplicate_idx].dropna()
        return X2.set_index(np.arange(X2.shape[0]))

In [ ]:
# text cleaner
class TextCleaner(BaseEstimator, TransformerMixin):
    # Transformer to remove punctuation and multiple spaces from text and change uppercase to lowercase

    def __init__(self,pattern="[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]"):
        self.pattern= pattern
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        X2= X.copy()
        X2.replace({"\s\s+":" "}, regex=False, inplace=True)

        for col in X2.columns:
            if X2.loc[:,col].dtypes == int: continue
            X2.loc[:,col] = X2.loc[:,col].str.replace(self.pattern,"", regex=True).str.lower()
        return X2

In [ ]:
class StopWordsRemover(BaseEstimator,TransformerMixin):
    # Transformer to remove popular english words with some default exceptions. User can add his own words to keep.
    # This is basically done to ensure keywords remain mostly in the reviews
    def __init__(self,words_to_keep = ['few','not','off','all','any','not','no','very']):
        stop_words = set(stopwords.words('english'))
        self.eng_words = stop_words.difference(set(words_to_keep))

    def fit(self,X,y=None):
        return self

    def transform(self,X):
        X2 = X.copy()

        for col in X2.columns:
            if X2.loc[:, col].dtypes == int:continue
            for en,review in enumerate(X2.loc[:,col].astype(str)):
                new = (" ").join(j for j in review.split(" ") if j.lower() not in self.eng_words)
                try:
                    X2.loc[:,col].iloc[en] = new
                except:
                    continue
            

        return X2

In [ ]:
# bring down keywords to their root by stemming
class Stemmer(BaseEstimator,TransformerMixin):
    # Transformer to stem words.
    def __init__(self, stem=True):
        self.stemmer = nltk.PorterStemmer()
        self.stem = stem
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if self.stem == False:
            return X
        else:
            X2 = X.copy()  
            for col in X2.columns:
                if X2.loc[:, col].dtypes == int: continue
                for en, review in enumerate(X2.loc[:, col].astype(str)):
                    new = (" ").join(self.stemmer.stem(j) for j in review.split(" "))
                    try:
                        X2[:, col].iloc[en] = new
                    except:
                        continue
            return X2


In [ ]:
class Rating(BaseEstimator, TransformerMixin):
    # Transformer to change reviews rating number to positive, negative and neutral
    # -1 for negative, 0 for neutral and 1 for positive

    def __init__(self, scale={1:-1, 2:-1, 3:0, 4:1, 5:1}, labels_to_del=[]):
        self.scale = scale
        self.labels_to_del = labels_to_del
    
    def fit(self, X, y=None):
        if self.labels_to_del != []:
            self.idx_to_del = X['rating'] == self.labels_to_del[0]
        return self

    def transform(self, X):    
        X2 = X.copy()
        if self.labels_to_del:
            X2 = X2[~self.idx_to_del]
        X2.replace(self.scale, inplace=True)
        return X2

In [ ]:
cols = ['review title', 'review content', 'rating']

#pipeline
preprocessor = Pipeline([
    #at first duplicated reviews will be removed
    ('DuplicateRemover', DuplicatesRemover()),
    #symbols that will be removed are defined in the transformer but a user can define his own/some additional symbols
    ('TextCleaning',TextCleaner()),
    #removing popular english words
    ('StopWordsRemover',StopWordsRemover()),
    #if stem is False the words will not be stemmed
    ('Stemmer', Stemmer(stem=False)),
    #rating changer, in this example negative(1, 2) ratings are equal to -1, neutral (3) 0 and positive(4,5) 1
    ('Rating', Rating(scale={1:-1, 2:-1, 3:0, 4:1, 5:1})),
    #the autor noticed that after cleaning the reviews some duplicated reviews are left, one more time duplicateremover is used (we could use it only one time, but it would make the process of data cleaning longer)
    ('DuplicateRemover2', DuplicatesRemover())
])

In [ ]:
#cleaned reviews are available after running this cell (note: it may take a while)
preprocessor.fit(raw_data[cols])
data_preprocessed = preprocessor.transform(raw_data[cols])

In [ ]:
data_preprocessed.head()

#### Approach 1
Weighted sum basic approach

* remove stop words, bring back each word via stemmer and check net-rating of the sentence.



In [ ]:
# creating a list of positive, negative and neutral keywords
with open('/home/suraj/ClickUp/Jan-Feb/DataScience_ML_DL_Projects/MachineLearning/Ecommerce_Product_Review/sentiment_analysis/data/positive-words.txt', 'r') as f:
    positive_words = [line.strip() for line in f]
with open('/home/suraj/ClickUp/Jan-Feb/DataScience_ML_DL_Projects/MachineLearning/Ecommerce_Product_Review/sentiment_analysis/data/negative-words.txt', 'r') as f:
    negative_words = [line.strip() for line in f]
with open('/home/suraj/ClickUp/Jan-Feb/DataScience_ML_DL_Projects/MachineLearning/Ecommerce_Product_Review/sentiment_analysis/data/neutral-words.txt', 'r') as f:
    neutral_words = [line.strip() for line in f]

In [ ]:
print(positive_words)

In [ ]:
print(negative_words)

In [ ]:
print(neutral_words)

In [ ]:
# taking a random review and calculating its sentiment
# review taken from - https://www.amazon.in/Top-Examples-Use-Essay-Evidence/product-reviews/1479248738
review = """This is a useful book for SAT practice. My daughter was able to increase her SAT score significantly by understanding how the SATs are reviewed and graded. It's not a short cut but it does help practice the test and allow you to focus on what's important. It is beyond my understanding why the school system in our area doesn't use such a tool rather than leaving it to the kids to have to navigate this area on their own. This is not cheating, it simply explains what the examiners are looking for."""

In [ ]:
# creating a function that can preprocess a paragraph of review by cleaning,removing stop words, stemming etc.
# Steps >>
# 1. Duplicate remover
# 2. Text cleaning
# 3. Stop word remover
# 4. Stemmer and lemmatizer
from collections import Counter
import re
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# This is a helper function to map NTLK position tags
# Full list is available here: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

#function to remove duplicates
def remove_duplicate(review):
    review = review.split(" ")
    #Unique words
    unique_words = Counter(review)
    # joining two adjacent element in iterable way
    s = " ".join(unique_words.keys())
    return s

#function to clean text
def clean_text(review):
    #converting to lowercase
    text = review.lower()
    # removing unicode characters.
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    #removing stopwords
    stop = stopwords.words('english')
    text = " ".join([word for word in text.split() if word not in (stop)])
    # stemming and lemmatization
    snow = SnowballStemmer('english')
    stemmed_sentence = []
    # Word Tokenizer
    words = word_tokenize(text)
    for w in words:
        #Apply stemming
        stemmed_sentence.append(snow.stem(w))
    stemmed_text = " ".join(stemmed_sentence)
    #lemmatization
    # Initialize the lemmatizer
    wl = WordNetLemmatizer()
    lemmatized_sentence = []

    words = word_tokenize(stemmed_text)
    word_pos_tags = nltk.pos_tag(words)
    # Get position tags
    word_pos_tags = nltk.pos_tag(words)
    # Map the position tag and lemmatize the word/token
    for idx, tag in enumerate(word_pos_tags):
        lemmatized_sentence.append(wl.lemmatize(tag[0], get_wordnet_pos(tag[1])))
    lemmatized_text = " ".join(lemmatized_sentence)
    return lemmatized_text




In [ ]:
# creating predict_function
def predict_sentiment(review):
    #removing duplicate
    review=remove_duplicate(review)
    review= clean_text(review)
    total_review_rating = []
    words = review.split()
    for word in words:
        if word in negative_words:
            total_review_rating.append(-1)
        elif word in positive_words:
            total_review_rating.append(1)
        elif word in neutral_words:
            total_review_rating.append(0)
    print(total_review_rating)
    net_rating = sum(total_review_rating)
    if net_rating ==0:
        return "neutral"
    elif net_rating >0:
        return "positive"
    else:
        return "negative"




In [ ]:
#predict
print(predict_sentiment(review))

In [ ]:
# review linkk - https://www.amazon.in/gp/customer-reviews/RIKJKSB3IR6LP?ASIN=B0006U7FC0review2 = """This product stopped working after just used for 3-4 times.
review2= """It is really worthless to buy such a costly product to waste money.
Neither seller is helping nor Amazon is helping me to get it services. I am not able to find a service center in Hyderabad."""

In [ ]:
print(predict_sentiment(review2))

In [ ]:
# link - https://www.amazon.in/product-reviews/B01F8YGEXE
review3 = """Liked the product. Cushioning is good. Some what small. But then it is 3ft by 6ft. Which is good for 1 person to sleep.
It is very comfortable. Has cotton cover. But its not waterproof. Hence better to put waterproof sheet over if using for an infant.
Overall good product and recommended."""

In [ ]:
print(predict_sentiment(review3))

# Conclusion of first method 
First method of depicting sentiment is done and the results are subject to keywords in the three files(positive-words.txt, negative-words.txt and neutral-words.txt) which right now look satisfactory

#### 2. Count Vectorizer with Multinomial Naive Bayes



In [ ]:
# removing any neutral ratings equal to 0 from the database
df = data_preprocessed[data_preprocessed['rating']!=0]

In [ ]:
df

In [ ]:
# train test split
# splitting the data into training set and validation
from sklearn.model_selection import train_test_split, GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(df['review content'], df['rating'], \
                                                    test_size=0.1, random_state=0)

print('Load %d training examples and %d validation examples. \n' %(X_train.shape[0],X_test.shape[0]))


In [ ]:
# Train MultinomialNB classifier


countVect = CountVectorizer() 
X_train_countVect = countVect.fit_transform(X_train)
mnb = MultinomialNB()
mnb.fit(X_train_countVect, y_train)

In [ ]:
def modelEvaluation(predictions):
    '''
    Print model evaluation to predicted result 
    '''
    print ("\nAccuracy on validation set: {:.4f}".format(accuracy_score(y_test, predictions)))
    print("\nAUC score : {:.4f}".format(roc_auc_score(y_test, predictions)))
    print("\nClassification report : \n", metrics.classification_report(y_test, predictions))
    print("\nConfusion Matrix : \n", metrics.confusion_matrix(y_test, predictions))

In [ ]:
# Evaluate the model on validaton set
predictions = mnb.predict(countVect.transform(X_test))
modelEvaluation(predictions)

#### Conclusion
Results of Count Vect with Multinomial naive bayes looks good!

#### 3. TFIDF vectorizer with Logistic regression.

In [ ]:
tfidf = TfidfVectorizer(min_df=5) #minimum document frequency of 5
X_train_tfidf = tfidf.fit_transform(X_train)
print("Number of features : %d \n" %len(tfidf.get_feature_names())) #1722
print("Show some feature names : \n", tfidf.get_feature_names()[::1000])

# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train_tfidf, y_train)

In [ ]:
# Look at the top 10 features with smallest and the largest coefficients
feature_names = np.array(tfidf.get_feature_names())
sorted_coef_index = lr.coef_[0].argsort()
print('\nTop 10 features with smallest coefficients :\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Top 10 features with largest coefficients : \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

In [ ]:
# Evaluate on the validaton set
predictions = lr.predict(tfidf.transform(X_test))
modelEvaluation(predictions)

## Conclusion
Results of TFIDF with Logistic regression looks much better.


#### 4. Using LSTM.

In this, we will use the raw_data which is not cleaned to train LSTM models and see how they perform with respect to the machine learning models created with hand crafted features.

In [ ]:
#dropping all 3 ratings from raw_data at first
df = raw_data[raw_data['rating']!=3]

In [ ]:
df

In [ ]:
# converting the ratings to boolean now via transformers pipeline
cols = ['review title', 'review content', 'rating']

#pipeline
preprocessor = Pipeline([
    #at first duplicated reviews will be removed
    #('DuplicateRemover', DuplicatesRemover()),
    #symbols that will be removed are defined in the transformer but a user can define his own/some additional symbols
    #('TextCleaning',TextCleaner()),
    #removing popular english words
    #('StopWordsRemover',StopWordsRemover()),
    #if stem is False the words will not be stemmed
    #('Stemmer', Stemmer(stem=False)),
    #rating changer, in this example negative(1, 2) ratings are equal to -1, neutral (3) 0 and positive(4,5) 1
    ('Rating', Rating(scale={1:-1, 2:-1, 3:0, 4:1, 5:1})),
    #('DuplicateRemover2', DuplicatesRemover())
])

In [ ]:
#cleaned reviews are available after running this cell (note: it may take a while)
preprocessor.fit(df[cols])
lstm_data = preprocessor.transform(df[cols])

In [ ]:
lstm_data

In [ ]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(lstm_data['review content'], lstm_data['rating'], \
                                                    test_size=0.1, random_state=0)

print('Load %d training examples and %d validation examples. \n' %(X_train.shape[0],X_test.shape[0]))


In [ ]:
top_words = 20000 
maxlen = 100 
batch_size = 32
nb_classes = 2
nb_epoch = 3


# Vectorize X_train and X_test to 2D tensor
tokenizer = Tokenizer(nb_words=top_words) #only consider top 20000 words in the corpse
tokenizer.fit_on_texts(X_train)
# tokenizer.word_index #access word-to-index dictionary of trained tokenizer

sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)

X_train_seq = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test_seq = sequence.pad_sequences(sequences_test, maxlen=maxlen)


# one-hot encoding of y_train and y_test
y_train_seq = np_utils.to_categorical(y_train, nb_classes)
y_test_seq = np_utils.to_categorical(y_test, nb_classes)

print('X_train shape:', X_train_seq.shape) #(27799, 100)
print('X_test shape:', X_test_seq.shape) #(3089, 100)
print('y_train shape:', y_train_seq.shape) #(27799, 2)
print('y_test shape:', y_test_seq.shape) #(3089, 2)